In [1]:

import os

from src.data_processing.data_loader import DataLoader
data_path=os.environ.get('DATA_PATH')

data=DataLoader(data_path).pre_process()
data['AttributeKey'].value_counts()

[INFO] Loading used combinations data...
[INFO] Loading descriptions data...
[INFO] Loading attributes data...
C:\Users\Public\anaconda\lib\site-packages\pandas\core\reshape\merge.py:1204: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():


Farbe                              13390
Beschreibung der Besonderheiten     8652
Produkttypbezeichnung               7989
Breite                              7096
Höhe                                6821
                                   ...  
Durchmesser des Kopfes                10
erf. Dauer des Vorheizens             10
Anzahl Variablenspeicher              10
Umweltfreundliches Merkmal            10
Verwendung für Einstecktasche          3
Name: AttributeKey, Length: 348, dtype: int64

In [2]:
from src.utils.data_transformations  import dataframe_for_two_attributes

df_mtpl_attributes=dataframe_for_two_attributes()

C:\Users\Public\anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
from src.utils.helpers import is_value_in_text

df=df_mtpl_attributes[df_mtpl_attributes.apply(lambda x: is_value_in_text(x['AttributeValue1'],x['ProductDescription']) and is_value_in_text(x['AttributeValue2'],x['ProductDescription']),axis=1)]

In [54]:
count_threshold = 10

df_filtered = df[df.groupby('AttributeKey1')['AttributeKey1'].transform('size') >= count_threshold]
df_filtered = df_filtered[df_filtered.groupby('AttributeKey2')['AttributeKey2'].transform('size') >= count_threshold]

In [47]:
df_filtered[['AttributeKey1','AttributeKey2']].value_counts()

AttributeKey1                             AttributeKey2                    
Anzahl der Farben                         Farbe                                2078
Ausführung der Griffzone                  Ausführung der Spitze                 588
Anzahl der Blätter                        Ausführung der Bindung                434
Anzahl der Seiten                         Ausführung der Kalenderaufteilung     423
Ausführung der Feder                      Ausführung der Griffzone              409
                                                                               ... 
Beschreibung der Besonderheiten           Bezeichnung der Härte                   1
Anzahl der Fächer                         Durchmesser                             1
Beschreibung der Besonderheiten           Farbe des Rahmens                       1
Bezeichnung der Härte                     Durchmesser                             1
Ausführung der Oberflächenbeschaffenheit  Farbe                                   1


In [56]:
df_filtered=df_filtered.query("`AttributeKey1` in ['Anzahl der Farben','Ausführung der Griffzone']")

In [58]:
df_filtered = df_filtered[df_filtered.groupby('AttributeKey2')['AttributeKey2'].transform('size') >= count_threshold]
df_filtered['AttributeKey2'].value_counts()

Farbe                              2078
Ausführung der Spitze               588
Ausführung des Inhalts (Set)        200
Ausführung der Vorschubmechanik     181
Ausführung des Behälters             38
Ausführung                           22
Name: AttributeKey2, dtype: int64

In [59]:
import pandas as pd
from sklearn.model_selection import train_test_split


train_data_list = []
test_data_list = []


attribute_columns = ['AttributeKey1', 'AttributeKey2']

for attribute_column in attribute_columns:
    attribute_key = attribute_column
    attribute_value = 'AttributeValue' + attribute_column[-1]
    train_data_attr, test_data_attr = train_test_split(df_filtered[df_filtered[attribute_key].notnull()], test_size=0.2, random_state=42, stratify=df_filtered[attribute_key])


    train_data_list.append(train_data_attr)
    test_data_list.append(test_data_attr)


train_data = pd.concat(train_data_list)
test_data = pd.concat(test_data_list)


print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)

Train data shape: (4970, 6)
Test data shape: (1244, 6)


In [62]:
test_data['AttributeKey2'].value_counts()

Farbe                              831
Ausführung der Spitze              239
Ausführung des Inhalts (Set)        83
Ausführung der Vorschubmechanik     69
Ausführung des Behälters            14
Ausführung                           8
Name: AttributeKey2, dtype: int64

In [7]:
training_data = []
cols=[('AttributeKey1','AttributeValue1'),('AttributeKey2','AttributeValue2')]

for _, row in train_data.iterrows():
    text = row["ProductDescription"]
    entities=[]
    for key,att in cols:
        attribute = row[att]
        label=row[key]
        start_index = text.lower().find(attribute.lower())
        if start_index != -1:
            end_index = start_index + len(attribute)
            entities.append((start_index, end_index, label))
    training_data.append((text, {"sc":entities}))

In [8]:
import spacy
from spacy.pipeline.spancat import DEFAULT_SPANCAT_MODEL
from spacy import displacy
from spacy.util import minibatch, compounding
from tqdm import tqdm
spacy.load('en_core_web_sm')
labels=train_data['AttributeKey1'].unique()

C:\Users\Public\anaconda\lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.6.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [9]:
config = {
    #this refers to the minimum probability to consider a prediction positive
    "threshold": 0.5,
    #the span key refers to the key in doc.spans
    "spans_key": 'entities',
    #this refers to the maximum number of labels to consider positive per span
    "max_positive": None,
     #a model instance that is given a list of documents with start end indices representing the labelled spans
    "model": DEFAULT_SPANCAT_MODEL,
    #A function that suggests spans. This suggester is fixed n-gram length of up to 3 tokens
    #"suggester": {"@misc": "spacy.ngram_suggester.v1", "sizes": [1, 2, 3]},
}

In [10]:
nlp=spacy.blank('de')
nlp.add_pipe("spancat", config=config)
span=nlp.get_pipe('spancat')
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "spancat"]
for label in labels:
    span.add_label(label)
nlp.initialize()
sgd = nlp.create_optimizer()

[2023-08-12 10:48:21,433] [INFO] Created vocabulary
[INFO] Created vocabulary
[2023-08-12 10:48:21,434] [INFO] Finished initializing nlp object
[INFO] Finished initializing nlp object


In [28]:
training_data[0][0]

'Kreidemarker Keilspitze 5-15mm FRANKEN ZKM1503 Kreidemarker Jumbo. Bezeichnung der Schreibflüssigkeit: schnell trocknende Flüssigkeit auf Wasserbasis. Ausführung der Spitze: Keilspitze. Hoche Farbintensität, gute Deckkraft, geeignet für Glas, Kreidetafeln, etc.. Material des Schaftes: Kunststoff. Kappe umsteckbar. Farbe des Schaftes: weiß. Strichstärke: 5-15.'

In [38]:
import spacy

def find_span_token_indices(text, span):

    nlp = spacy.load("de_core_news_sm")
    doc = nlp(text)
    span_tokens = span.split()
    start_token_index = None
    end_token_index = None
    for i, token in enumerate(doc):
        if token.text == span_tokens[0] and i + len(span_tokens) <= len(doc):
            match = True
            for j in range(1, len(span_tokens)):
                if doc[i + j].text != span_tokens[j]:
                    match = False
                    break
            if match:
                start_token_index = i
                end_token_index = i + len(span_tokens) - 1

    return (start_token_index, end_token_index)

# Example usage
text = "Welcome to the Bank of China."
span1 = "China"
span2 = "Bank of China"


indices1 = find_span_token_indices(text, span1)
indices2 = find_span_token_indices(text, span2)


print(f"Span '{span1}': {indices1}")
print(f"Span '{span2}': {indices2}")


Span 'China': (5, 5)
Span 'Bank of China': (3, 5)


In [44]:
len(train_data)

20178

In [41]:
train_data['ProductDescription'].iloc[0]

'Kreidemarker Keilspitze 5-15mm FRANKEN ZKM1503 Kreidemarker Jumbo. Bezeichnung der Schreibflüssigkeit: schnell trocknende Flüssigkeit auf Wasserbasis. Ausführung der Spitze: Keilspitze. Hoche Farbintensität, gute Deckkraft, geeignet für Glas, Kreidetafeln, etc.. Material des Schaftes: Kunststoff. Kappe umsteckbar. Farbe des Schaftes: weiß. Strichstärke: 5-15.'

In [42]:
train_data['AttributeValue1'].iloc[0]

'Keilspitze'

In [40]:
find_span_token_indices(train_data['ProductDescription'].iloc[0],train_data['AttributeValue1'].iloc[0])

(25, 25)

In [63]:
training_data = []
cols = [('AttributeKey1', 'AttributeValue1'), ('AttributeKey2', 'AttributeValue2')]

for i, row in train_data.iterrows():
    print(i)
    text = row["ProductDescription"]
    entities = []
    for key, att in cols:
        attribute = row[att]
        label = row[key]
        start_index, end_index=find_span_token_indices(text,attribute)
        entities.append((start_index, end_index, label))
    training_data.append((text, {"sc": entities}))

6916
11847
4437
11026
17636
7661
6015
2199
6949
559
5961
1957
6948
3909
22028
216
8720
5570
11205
37
796
13853
2221
6913
7666
5938
2803
2120
5007
11676
10094
5419
12209
19264
23740
22184
14886
1642
2044
21015
6222
1616
5911
19593
14503
14228
11481
4810
17875
20299
8240
22179
2166
10197
5420
1640
21018
14533
20976
5982
12762
2204
22173
137
2365
1622
12139
21462
18583
5948
10982
1956
3373
1187
356
3272
1128
2350
17517
371
393
17571
14076
21595
20167
5970
5523
12210
386
9664
8628
12271
21006
2130
11256
15294
21155
5192
209
5421
2080
5188
3280
5995
3203
21092
5913
23140
2148
12274
8630
5433
23690
5435
21786
1688
11090
3675
19626
22114
3204
15024
21032
2465
7710
9826
798
4232
9400
7293
9848
1686
206
2357
2917
8553
23177
12261
9847
2205
18269
1529
1667
331
18271
22194
234
5638
4916
21046
14467
5999
8580
5509
22199
18446
5560
11208
799
4920
5539
20066
5513
1921
1627
10403
8582
5916
17857
8241
14552
10570
2168
18529
8941
3458
3287
5626
20175
23192
20340
7712
1648
4811
1621
11983
5644
12273
710

KeyboardInterrupt: 

In [70]:
 training_data[0]

('Kugelschreiber Einweg super soft grün   K86s PELIKAN 804400 Kugelschreiber Stick K86s super soft grün. Kappenmodell. Typbezeichnung der Mine: Einweg-Kugelschreibermine. Ausführung Mine: M, grün. Ausführung Stift: Kunststoff. Clip: Kunststoff, in Schreibfarbe. Dreieckige Form, Soft-Writing. Farbe des Schaftes: transparent. Ausführung des Inhalts mit Packung: 12 Stück in Faltschachtel.',
 {'sc': [(38, 38, 'Ausführung der Griffzone'),
   (16, 16, 'Ausführung der Vorschubmechanik')]})

In [96]:
training_data[7][1]

{'sc': [(47, 47, 'Ausführung der Griffzone'),
  (18, 18, 'Ausführung der Spitze')]}

In [97]:
from spacy.training import Example
doc= nlp.make_doc(training_data[7][0])
annotation={'spans': {'sc': [(7, 7, 'Ausführung der Griffzone'),
  (10, 10, 'Ausführung der Spitze')]}}
example=Example.from_dict(doc, annotation)
example

ValueError: [E855] Invalid span: span is not from the same doc.

In [69]:
from spacy.training import Example

data_train=[]
for raw_text, entity_offsets in training_data:
    print(raw_text)
    print(entity_offsets)
    doc = nlp.make_doc(raw_text)
    annotation = {'spans':entity_offsets}
    data_train.append(Example.from_dict(doc, annotation))

Kugelschreiber Einweg super soft grün   K86s PELIKAN 804400 Kugelschreiber Stick K86s super soft grün. Kappenmodell. Typbezeichnung der Mine: Einweg-Kugelschreibermine. Ausführung Mine: M, grün. Ausführung Stift: Kunststoff. Clip: Kunststoff, in Schreibfarbe. Dreieckige Form, Soft-Writing. Farbe des Schaftes: transparent. Ausführung des Inhalts mit Packung: 12 Stück in Faltschachtel.
{'sc': [(38, 38, 'Ausführung der Griffzone'), (16, 16, 'Ausführung der Vorschubmechanik')]}


ValueError: [E855] Invalid span: span is not from the same doc.

In [ ]:
import random

all_losses = []
with nlp.disable_pipes(*other_pipes):
    for iteration in tqdm(range(30)):
        # shuffling examples before every iteration
        random.shuffle(train_data)
        losses = {}
        batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            nlp.update(list(batch), losses=losses, drop=0.1, sgd=sgd)
        print("epoch: {} Losses: {}".format(iteration, str(losses)))
        all_losses.append(losses['spancat'])
    nlp.to_disk()
